In [1]:
import leaderboard

In [2]:
leaderboard.data

,1,2,3,4,5,6,7
2316524,3353,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2728245,953,868,<NA>,<NA>,<NA>,<NA>,<NA>
740740,1001,976,7452,325,47268,148,<NA>
999754,434,1572,680,750,770,302,<NA>
Alex Ji,914,530,441,<NA>,<NA>,<NA>,<NA>
attoPascal,117,513,444,267,394,40,247
Azratosh,120,103,247,181,456,49,1084
benjo121ben,419,952,222,1083,325,107,1520
Benni S,475,788,2688,276,628,1593,417
christoph-det,327,1989,4482,166,988,140,370


In [3]:
leaderboard.points().head(20)

,pts,name
1,336,Azratosh
2,327,msonnberger
3,326,TechnikTobi
4,325,attoPascal
5,316,Grarak
6,314,RaphaelTarita
7,314,David Mikan
8,313,felpower
9,308,devgertschi
10,307,Hainkinho


In [4]:
leaderboard.seconds_latest_day()

,7
felpower,181
attoPascal,247
msonnberger,278
Grarak,368
christoph-det,370
Benni S,417
JohnBryte,419
Salominius,422
devgertschi,466
NotsoblackReaper,474


In [5]:
leaderboard.seconds_all_days() // 60

,0
msonnberger,28
attoPascal,33
TechnikTobi,34
devgertschi,36
Azratosh,37
NotsoblackReaper,37
felpower,39
Grarak,42
thoosequa,42
Salominius,44
